In [ ]:
import numpy as np, os, random, librosa
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
dir =  # where the wav's are
data = # where to save the cqt's

key = # name of .npy file
'
FOLDS = 5 # folds for cross validation

In [ ]:
x, y = [[] for _ in range(FOLDS)], [[] for _ in range(FOLDS)]

for d in os.listdir(dir):
    if not os.path.isdir(dir+d): continue
    
    x_temp = [dir+d+'/'+f for f in os.listdir(dir+d)]
    random.shuffle(x_temp)
    
    split = int(np.floor(len(x_temp)//FOLDS))
    for i in range(FOLDS):
        x[i].extend(x_temp[i*split:(i+1)*split])
        y[i].extend([d]*split)

for i in range(FOLDS):
    mlb = MultiLabelBinarizer()
    p = np.random.permutation(len(x[i]))
    x[i] = np.array(x[i])[p]
    y[i] = np.array(y[i])[p]
    y[i] = [label.split('-') for label in y[i]]
    y[i] = mlb.fit_transform(y[i])

In [ ]:
def cqt(x):
    X_train = []
    for f in x:
        wav,sr = librosa.load(f,mono=True,sr=22050)
        wav = wav / np.sqrt(np.mean(wav**2))
        cqt = librosa.core.cqt(wav,n_bins=96,bins_per_octave=12,hop_length=256)
        X_train.append(librosa.core.amplitude_to_db(np.abs(cqt)))      
    return np.expand_dims(np.array(X_train),-1)

In [ ]:
for i in range(FOLDS): 
    np.save(data+'x_fold'+str(i)+'_'+key+'.npy',cqt(x[i]))
    np.save(data+'y_fold'+str(i)+'_'+key+'.npy',y[i]); 
    print("Created fold "+str(i)+".")